In [104]:
import ee
import geemap
# import geemap.colormaps as cm

try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


In [64]:
region = ee.Geometry.Polygon([
  [
    [10.7, 50.3],
    [10.7, 50.471],
    [10.9, 50.471],
    [10.9, 50.3],
    [10.7, 50.3] 
  ]
])



In [ ]:
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)
    
years = [2017, 2018, 2019, 2020]
ndvi_images = {}

for year in years:
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(region)
        .filterDate(f'{year}-07-01', f'{year}-09-30')
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
        .map(calculate_ndvi)
    )
    print(f'Number of images in the collection {year}: ', len(collection.aggregate_array('system:index').getInfo()))
    ndvi_images[year] = collection.median().clip(region)

ndvi_diff_2018_2017 = ndvi_images[2018].select('NDVI').subtract(ndvi_images[2017].select('NDVI'))
ndvi_diff_2019_2018 = ndvi_images[2019].select('NDVI').subtract(ndvi_images[2018].select('NDVI'))
ndvi_diff_2020_2019 = ndvi_images[2020].select('NDVI').subtract(ndvi_images[2019].select('NDVI'))

nri = (ndvi_diff_2020_2019.add(ndvi_diff_2019_2018).add(ndvi_diff_2018_2017)).divide(ndvi_images[2017].select('NDVI')).rename('NRI')

training = nri.sample(region, scale=20, numPixels=5000)
clusterer = ee.Clusterer.wekaKMeans(3).train(training)
nri_clustered = nri.cluster(clusterer).clip(region)

In [123]:
palette = cm.get_palette("inferno", num_clusters) 
cluster_vis = {
    'min': 0,
    'max': num_clusters - 1,  
    'palette': palette
}

s2_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}

aster_vis = {
    'bands': ['B02', 'B01'],
    'min': 0,
    'max': 255
}
nri_vis = {
    'min': -1,  
    'max': 1,   
    'palette': ['red', 'white', 'green']
}
ndvi_vis = {'min': -0.2, 'max': 0.8, 'palette': ['blue', 'white', 'green']}



Map2 = geemap.Map()
Map2.addLayer(nri, nri_vis, 'NDVI Recovery Index (NRI)')
Map2.addLayer(ndvi_images[2017],s2_vis,'Satelite 2017')
Map2.addLayer(ndvi_images[2018],s2_vis,'Satelite 2018')
Map2.addLayer(ndvi_images[2019],s2_vis,'Satelite 2019')
Map2.addLayer(ndvi_images[2020],s2_vis,'Satelite 2020')
Map2.addLayer(nri_clustered, cluster_vis, 'Field Clusters')
Map2.setCenter(10.8, 50.35, 11)
Map2

Map(center=[50.35, 10.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [115]:
unique_clusters = nri_clustered.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=region,
    scale=30,
    maxPixels=1e8
).get('cluster')

# Inspect the unique cluster distribution (for debugging)
print("Unique Cluster Distribution: ", unique_clusters.getInfo())

Unique Cluster Distribution:  {'0': 112830.61960784336, '1': 18273.399999999987, '2': 339868.76470588}


In [43]:

# aster = ee.ImageCollection("ASTER/AST_L1T_003") \
#     .filterBounds(region) \
#     .filterDate('2018-06-01', '2018-08-30')

In [44]:
listOfImages = aster.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', len(listOfImages))

Number of images in the collection:  18
Number of images in the collection:  3


In [45]:
aster_img = aster.mosaic().clip(region)

In [47]:
def calculate_evi(image):
    evi = image.expression(
        '2.5 * (NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1)', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }
    ).rename('EVI')
    return image.addBands(evi)

In [54]:
sentinel2_img_features = calculate_ndvi(sentinel2_img)
sentinel2_img_features = calculate_evi(sentinel2_img_features)

feature_bands = ['B2', 'B3', 'B4', 'B8', 'NDVI', 'EVI']
feature_image = sentinel2_img_features.select(feature_bands)

training_data = feature_image.sample(
    region=region,
    scale=10,
    numPixels=5000,  
    geometries=True
).randomColumn()

clusterer = ee.Clusterer.wekaKMeans(5).train(training_data.select(feature_bands),feature_bands)
clustered_image = feature_image.cluster(clusterer).rename('clusters')